In [1]:
print("ok")

ok


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [4]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [23]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-005")
from langchain_google_genai import ChatGoogleGenerativeAI
llm_google = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

In [7]:
from langchain_groq import ChatGroq
llm_gorq=ChatGroq(model="mixtral-8x7b")
llm_gorq

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000229A9EC0370>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000229A9E97D30>, model_name='mixtral-8x7b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm_google.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Please provide me with more context! I need to know what you'd like me to do. For example, are you asking me to:

*   **Answer a question?** If so, what is the question?
*   **Write something?** If so, what should I write about?
*   **Summarize something?** If so, what should I summarize?
*   **Translate something?** If so, what should I translate and into what language?
*   **Generate code?** If so, what kind of code and what should it do?
*   **Give you information about a specific topic?** If so, what is the topic?

The more information you give me, the better I can assist you.


In [8]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [9]:
store={}

In [11]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [12]:
config = {"configurable": {"session_id": "firstchat"}}

In [13]:
model_with_memory=RunnableWithMessageHistory(llm_google,get_session_history)

In [14]:
model_with_memory.invoke(("Hi! I'm Aniruddha Mohanty"),config=config).content

"Hi Aniruddha! It's nice to meet you. How can I help you today?"

In [15]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Aniruddha Mohanty.'

In [16]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Aniruddha Mohanty", additional_kwargs={}, response_metadata={}), AIMessage(content="Hi Aniruddha! It's nice to meet you. How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--67302a94-2eb9-47cb-99c7-01475c9ce99a-0', usage_metadata={'input_tokens': 11, 'output_tokens': 22, 'total_tokens': 33, 'input_token_details': {'cache_read': 0}}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Aniruddha Mohanty.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--8de91259-02cf-4bb4-9eb1-00936aec1ae4-0', usage_metadata={'input_tokens': 39, 'output_tokens': 11, 'total_tokens': 50, 'input_token_details': 

In [17]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [25]:
loader = DirectoryLoader('/data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

GoogleGenerativeAIError: Error embedding content: 404 models/text-embedding-005 is not found for API version v1beta, or is not supported for embedContent. Call ListModels to see the list of available models and their supported methods.

In [26]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [27]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

NameError: name 'retriever' is not defined

In [28]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

NameError: name 'retrieval_chain' is not defined

In [29]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [30]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [31]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [32]:
tool.name

'wikipedia'

In [33]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [34]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [35]:
tool.return_direct

False

In [36]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [37]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

In [38]:

from langchain_core.pydantic_v1 import BaseModel, Field

d:\PythonCode\Langgraph-End-To-End\langgraphvenv\lib\site-packages\IPython\core\interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [39]:
class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [40]:
tool=WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

ValidationError: 1 validation error for WikipediaQueryRun
args_schema
  Input should be a subclass of BaseModel [type=is_subclass_of, input_value=<class '__main__.WikiInputs'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.12/v/is_subclass_of